<a href="https://colab.research.google.com/github/priyalaggarwal/Metastases-Detection-in-Gigapixel-Images/blob/master/Metastases_Detection_in_Gigapixel_Images_(CAMELYON16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install setuptools==45.3.0

In [ ]:
!apt-get install openslide-tools
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide
from openslide import __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray